# Telegram-бот: кнопки

Когда мы отправляем пользователю сообщение с кнопками для ответа, мы отправляем несколько объектов:
* само сообщение;
* клавиатуру, в которой разместятся наши кнопки - и как часть этой клавиатуры сами кнопки.

Кнопка тоже состоит из элементов: видимый для пользователя элемент - это текст кнопки, то, что на ней написано. Но может запросто быть, что мы задаем пользователю несколько вопросов, на которые предполагаются одинаковые варианты ответа - например, "да" и "нет". И если мы получаем только текст с кнопки, то мы не можем никак узнать, на какой из вопросов мы получили ответ. Поэтому второй элемент каждой кнопки, который помогает нам получить больше информации об ответе пользователя - колбэк. При помощи колбэка мы можем определить, какую функцию запустить для обработки ответа, и передать дополнительную информацию, какую мы захотим. Эти два элемента - колбэк и текст кнопки передаются в кнопку при ее создании при помощи двух именованных аргументов.

```python
from my_token import TOKEN
import telebot

bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def start_command(message):
    bot.send_message(message.chat.id, 'Привет-привет. Как твои дела?')

@bot.message_handler(func=lambda m: True) # Клавиатуру с кнопками будем отправлять в ответ 
                                          # на сообщение пользователя
def handle_message(message):
    markup = telebot.types.InlineKeyboardMarkup() # Сначала создадим клавиатуру, 
                                                  # в которой будут содержаться наши кнопки.
    item_1 = telebot.types.InlineKeyboardButton(text='Кнопка 1', callback_data='button_1') # Создадим первую кнопку
    item_2 = telebot.types.InlineKeyboardButton(text='Кнопка 2', callback_data='button_2') # Создадим вторую кнопку
    markup.add(item_1, item_2) # Добавим обе созданные кнопки в созданную клавиатуру
    bot.send_message(message.chat.id, 'Выбери кнопку', reply_markup=markup) # Отправим пользователю сообщение 
                                                                            # с возможностью ответить на него 
                                                                            # при помощи кнопок.

```

Все хорошо, только эти кнопки не работают. На самом деле это не так. Они работают, просто мы не прописали в приложении действия, которые должны выполняться по нажатию на эти кнопки. Для этого введем новый слушатель, только в этот раз он будет ждать не сообщение, а конкретную команду с кнопки.

```python
from my_token import TOKEN
import telebot

bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def start_command(message):
    bot.send_message(message.chat.id, 'Привет-привет. Как твои дела?')

@bot.message_handler(func=lambda m: True)
def handle_message(message):
    markup = telebot.types.InlineKeyboardMarkup()
    item_1 = telebot.types.InlineKeyboardButton(text='Кнопка 1', callback_data='button_1')
    item_2 = telebot.types.InlineKeyboardButton(text='Кнопка 2', callback_data='button_2')
    markup.add(item_1, item_2)
    bot.send_message(message.chat.id, 'Выбери кнопку', reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data == 'button_1') # Слушатель команды - колбэка
def button_1(call): # call содержит все данные, которые мы получим от кнопки
    print(call) # Напечатаем эти данные и посмотрим, что в них содержится
    bot.send_message(call.message.chat.id, 'Вы нажали кнопку 1') # И отправим пользователю сообщение

bot.polling(none_stop=True)

```

Еще полезные штуки:

```python
from my_token import TOKEN
import telebot

bot = telebot.TeleBot(TOKEN)


def birthdate_reply(message):
    birthdate = message.text
    print(birthdate)


@bot.message_handler(commands=['start'])
def start_command(message):
    # ForceReply сделает, чтобы следующее сообщение от пользователя использовало цитирование сообщения от бота
    # input_field_placeholder добавит пример ввода данных - бледную строку в поле ввода, 
    #    которая отображается до тех пор, пока пользователь не начнет вводить данные
    markup = telebot.types.ForceReply(input_field_placeholder='01.01.1990', selective=False)
    msg = bot.send_message(message.chat.id, 'Введите вашу дату рождения', reply_markup=markup)
    # register_next_step_handler определит, какая функция должна быть запущена при получении следующего сообщения пользоватля
    bot.register_next_step_handler(msg, birthdate_reply)

bot.polling(none_stop=True)
```

Это далеко не все возможности библиотеки. Если хотите использовать другие возможности, изучите ее **[документацию](https://pytba.readthedocs.io/en/latest/types.html#telebot.types.InlineKeyboardMarkup.max_row_keys)**. 

### Задача 1

В классе 12 двухместных парт. Бот спрашивает у пользователя количество учеников в классе. Если все ученики смогут разместиться за партами, бот должна сообщить об этом пользователю. Если парт слишком мало, бот просит пользователя выбрать класс побольше.

### Задача 2.1

Бот спрашивает у пользователя дату его рождения (формат ввода даты должен быть выведен в виде подсказки - плейсхолдера). Пользователь вводит свой день рождения. Если день и месяц совпадают с сегодняшними, бот поздравляет пользователя с днем рождения. Если нет, программа говорит, что запомнила дату.

### Задача 2.2 *

Помимо поздравления или сообщения об успешном запоминании даты бот должен отправить сообщение с возрастом пользователя.